In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Check current working directory
os.getcwd()

'C:\\Users\\krish\\Desktop\\All_ML\\fast_campus'

In [3]:
# 
df = pd.read_csv("Data/otto_train.csv") # 
df.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [5]:
# Extract x and y
nCar = df.shape[0]
nVar = df.shape[1]

df = df.drop("id", axis=1) # id is meaning less

In [7]:
# target
mapping_dict = {"Class_1": 1,
                "Class_2": 2,
                "Class_3": 3,
                "Class_4": 4,
                "Class_5": 5,
                "Class_6": 6,
                "Class_7": 7,
                "Class_8": 8,
                "Class_9": 9}
after_mapping_target = df['target'].apply(lambda x: mapping_dict[x])

In [9]:
feature_columns = list(df.columns.difference(["target"]))

x = df[feature_columns]
y = after_mapping_target

x_train,x_test, y_train, y_test = train_test_split(x, y,test_size = 0.2, 
                                                  random_state = 42)

print(x_train.shape, x_test.shape)

(49502, 93) (12376, 93)


## XGBoost

In [12]:
#! pip install xgboost

import xgboost as xgb
import time

start = time.time()
xgb_dtrain = xgb.DMatrix(data = x_train, label = y_train)
xgb_dtest = xgb.DMatrix(data=x_test)

xgb_param = {
    "max_depth":10, # tree depth
    "learning_rate":0.01, # Step size
    "n_estimators":100,# Number of tree
    "objective":"multi:softmax", # activation function
    "num_class": len(set(y_train)) + 1}

xgb_model = xgb.train(params=xgb_param, dtrain=xgb_dtrain)
xgb_model_predict = xgb_model.predict(xgb_dtest)

print("Accuravy ; %.2f"%(accuracy_score(y_test, xgb_model_predict)*100))
print("Time: %.2f"%(time.time() - start), "seconds") # 

[20:37:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:37:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuravy ; 76.67
Time: 2.43 seconds


## Light GBM

In [16]:
# pip install LightGBM

import lightgbm as lgb
start = time.time()
lgb_dtrain = lgb.Dataset(data=x_train, label = y_train)
lgb_param = {
    "max_depth":10, # Tree depth
    "learning_rate":0.1, # Step size
    "n_estimators":100, # Number of trees
    "objective":"multiclass",
    "num_class":len(set(y_train))+1}

lgb_model = lgb.train(params = lgb_param, train_set= lgb_dtrain)

lgb_model_predict = np.argmax(lgb_model.predict(x_test), axis = 1)

print("Accuracy: %.2f" %(accuracy_score(y_test, lgb_model_predict)*100))
print("Time: %.2f" %(time.time() - start), "seconds")

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005722 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3110
[LightGBM] [Info] Number of data points in the train set: 49502, number of used features: 93
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -3.476745
[LightGBM] [Info] Start training from score -1.341381
[LightGBM] [Info] Start training from score -2.039019
[LightGBM] [Info] Start training from score -3.135151
[LightGBM] [Info] Start training from score -3.125444
[LightGBM] [Info] Start training from score -1.481556
[Ligh

In [19]:
# Accuracy is increased up to 81
lgb_model.predict(x_test)[0].argmax()

2

## Catboost

In [25]:
# pip install catboost
import catboost as cb

start = time.time()
cb_dtrain = cb.Pool(data = x_train, label = y_train)
cb_param = {
    "max_depth":10,
    "learning_rate":0.01,
    "n_estimators":100,
    "eval_metric":"Accuracy",
    "loss_function":"MultiClass"
}

cb_model = cb.train(pool = cb_dtrain, params = cb_param)
cb_model_predict = np.argmax(cb_model.predict(x_test), axis = 1) + 1

print("Accuracy ", accuracy_score(y_test, cb_model_predict) * 100)
print("Time ", (time.time() - start))

0:	learn: 0.5907034	total: 393ms	remaining: 38.9s
1:	learn: 0.6356107	total: 638ms	remaining: 31.3s
2:	learn: 0.6411256	total: 891ms	remaining: 28.8s
3:	learn: 0.6480344	total: 1.25s	remaining: 30s
4:	learn: 0.6508222	total: 1.51s	remaining: 28.7s
5:	learn: 0.6499939	total: 1.76s	remaining: 27.5s
6:	learn: 0.6507818	total: 2.09s	remaining: 27.7s
7:	learn: 0.6548422	total: 2.34s	remaining: 26.9s
8:	learn: 0.6559533	total: 2.6s	remaining: 26.3s
9:	learn: 0.6560947	total: 2.84s	remaining: 25.6s
10:	learn: 0.6568421	total: 3.21s	remaining: 26s
11:	learn: 0.6588219	total: 3.46s	remaining: 25.4s
12:	learn: 0.6592259	total: 3.72s	remaining: 24.9s
13:	learn: 0.6611248	total: 4.01s	remaining: 24.7s
14:	learn: 0.6625591	total: 4.26s	remaining: 24.2s
15:	learn: 0.6631853	total: 4.51s	remaining: 23.7s
16:	learn: 0.6639328	total: 4.75s	remaining: 23.2s
17:	learn: 0.6668821	total: 5.11s	remaining: 23.3s
18:	learn: 0.6669630	total: 5.35s	remaining: 22.8s
19:	learn: 0.6675286	total: 5.59s	remaining: 2

In [26]:
cb_model.predict(x_test)

array([[-0.35426047,  1.22109587,  0.44230101, ..., -0.1698448 ,
        -0.02059177, -0.2130643 ],
       [-0.07235138,  0.42535181,  0.20060428, ...,  0.21863604,
         0.2719157 ,  0.25089315],
       [-0.3315885 , -0.31862353, -0.31279765, ..., -0.29798357,
        -0.24018767, -0.32984969],
       ...,
       [ 0.05304325,  0.02500267, -0.14752573, ..., -0.20741963,
         0.12789417,  1.51166757],
       [-0.55093666,  1.7691278 ,  0.99746884, ..., -0.3420542 ,
        -0.49799871, -0.38136323],
       [-0.3033724 ,  0.09352675, -0.11808658, ...,  0.65825036,
         1.05515787, -0.20799899]])

## Load Data

In [27]:
df = pd.read_csv("Data/kc_house_data.csv")
df.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [29]:
# Drop Un-necessary data
df = df.drop(["id", "date", "zipcode", "lat", 'long'], axis = 1)
df.head(2)

,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated
0,221900.0,3,1.00,1.0,0,3,7,1955,0
1,538000.0,3,2.25,2.0,0,3,7,1951,1991


In [ ]:
# Feature is Price , (y)
feature_columns = list(df.columns.difference(["price"])) #remove price

x = df[feature_columns]
